In [ ]:
model_path = "/content/drive/MyDrive/Interpretable Classifier Data/SemSeg Model/best_model.pt"
data_path = "/content/drive/MyDrive/Interpretable Classifier Data/Decision Tree Data"
save_path = "/content/drive/MyDrive/Interpretable Classifier Data/Decision Tree Model"

# Importing Libraries

In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import os
import cv2
import csv  
import graphviz
import pandas as pd
import numpy as np
import seaborn as sns
from joblib import dump
from model import ENet
from utils import load_ckp, decode_segmap, counter
from sklearn.tree import DecisionTreeClassifier, plot_tree, export_graphviz
from sklearn.metrics import classification_report, confusion_matrix

# Loading Model

In [ ]:
model = ENet(num_classes=3).to('cuda')

In [ ]:
# define optimzer
optimizer = torch.optim.Adam(model.parameters(),lr=5e-4,weight_decay=2e-4)

# define checkpoint saved path
ckp_path = model_path

In [ ]:
model, optimizer, start_epoch, valid_loss_min = load_ckp(ckp_path, model, optimizer)

In [ ]:
print("start_epoch = ", start_epoch)
print("valid_loss_min = ", valid_loss_min)

# Loading Data

In [ ]:
# Tranformation for the input images 
transform_img = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
  ])

In [ ]:
class dataset(Dataset):
  """Custom Dataset class for ease of operation"""

  def __init__(self, images_X, images_Y, transform_img):
    self.data=images_X
    self.labels=images_Y
    self.transform=transform_img

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    x = self.data[index]
    y = self.labels[index]
    x = self.transform(x)
    return x, y

In [ ]:
image_list_X_test = []

In [ ]:
num_bicycles = len(os.listdir(data_path+'/bicycle/'))

for imagename in sorted(os.listdir(data_path+'/bicycle/')): 
  im=cv2.imread(data_path+'/bicycle/'+imagename)
  image_list_X_test.append(im)

In [ ]:
num_unicycles = len(os.listdir(data_path+'/unicycle/'))

for imagename in sorted(os.listdir(data_path+'/unicycle/')): 
  im=cv2.imread(data_path+'/unicycle/'+imagename)
  image_list_X_test.append(im)

In [ ]:
num_negative = len(os.listdir(data_path+'/negative/'))

for imagename in sorted(os.listdir(data_path+'/negative/')): 
  im=cv2.imread(data_path+'/negative/'+imagename)
  image_list_X_test.append(im)

In [ ]:
images_X_test = np.array(image_list_X_test)
y = [2]*num_bicycles + [1]*num_unicycles + [0]*num_negative

In [ ]:
test_data = dataset(images_X=images_X_test, images_Y=y, transform_img=transform_img)
test_loader = DataLoader(test_data, batch_size=1)

# Creating CSV file

In [ ]:
# data rows of csv file  
rows = []  

In [ ]:
# field names  
fields = ['Num_Wheels', 'Num_Frames', 'Label']  
       
# name of csv file  
filename = "data.csv"

for img, label in test_loader:

  model.eval()
  
  xb = img.to('cuda')

  yb = model(xb)

  pred = F.softmax(yb, dim=1)                   
  preds = torch.argmax(pred, dim=1).squeeze(1)

  preds = preds.cpu()

  x = decode_segmap(preds,nc=3)
  x = x.squeeze(0)

  temp = counter(x)
  temp.append(int(label))
  rows.append(temp)

In [ ]:
# writing to csv file  
with open(filename, 'w') as csvfile:  
    # creating a csv writer object  
    csvwriter = csv.writer(csvfile)  
        
    # writing the fields  
    csvwriter.writerow(fields)  
        
    # writing the data rows  
    csvwriter.writerows(rows)

# Decision Tree

In [ ]:
data = pd.read_csv('data.csv')
y = data['Label']
data = data.drop(['Label'],axis=1)

In [ ]:
classifier = DecisionTreeClassifier(max_leaf_nodes=5, max_depth=4)
clf = classifier.fit(data,y)

In [ ]:
dot_data = export_graphviz(classifier, out_file=None,
                           feature_names=data.columns,  
                     class_names=['neither','unicycle','bicycle'],precision=1,rounded=True,filled=True) 
graph = graphviz.Source(dot_data)
graph

In [ ]:
print(confusion_matrix(y, clf.predict(data)))
print(classification_report(y, clf.predict(data)))

In [ ]:
cf = confusion_matrix(y, clf.predict(data))
sns.heatmap(cf, annot=True)

In [ ]:
# saving decision tree classifier
dump(clf, save_path + '/tree.joblib')